In [1]:
import numpy as np
from matplotlib import pyplot as plt

# import specpy as sp

from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator

from autosted.callback_buildingblocks.parameter_filtering import LocationKeeper, LocationRemover
from autosted.callback_buildingblocks.data_selection import NewestDataSelector, NewestSettingsSelector
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader, ScanModeSettingsGenerator, FOVSettingsGenerator
from autosted.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator, ScanOffsetsSettingsGenerator, MultipleScanOffsetsSettingsGenerator
from autosted.callback_buildingblocks.value_wrappers import BoundingBoxLocationGrouper
from autosted.callback_buildingblocks.repetition import ResultsRepeater

from autosted.detection.autofocus import SimpleFocusPlaneDetector
from autosted.detection.legacy import LegacySpotPairFinder, PairedLegacySpotPairFinder

from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import TimedStoppingCriterion

# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

In [ ]:
save_path = 'acquisition_data/threestep_spots_orthogonal_cuts'

# time to image in seconds
time_to_image = 1 * 60 * 60

# fov sizes and pixel sizes at various levels
# NOTE: may be set to None to just use values from settings file
fov_sizes = {"overview": [5e-5, 5e-5, 0.5e-5], "detail": [2e-6, 2e-6, 2e-6], "sted": [2.5e-7, 2.5e-7, 5e-7]}
pixel_sizes = {"overview": [3e-7, 3e-7, 4e-7], "detail": [0.8e-7, 0.8e-7, 1e-7], "sted": [5e-9, 5e-9, 5e-9]}

# how far to move in overview spiral (set larger than fov_size to have non-overlapping)
overview_spiral_move = [55e-6, 55e-6]

# 2-channel settings for levels overview + detail
overview_settings_file = 'C:/Users/RESOLFT/Desktop/config_json/20180706_overview_60x_2ch.json'
detail_settings_file = 'C:/Users/RESOLFT/Desktop/config_json/20180706_overview_60x_2ch.json'

# 6 single-channel settings for 3 orthogonal cuts in 2 channels 
sted_settings_files = [
    'C:/Users/RESOLFT/Desktop/config_json/20180706_2dsted_60x_ch2.json',
    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch2.json',
    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch2.json',
    'C:/Users/RESOLFT/Desktop/config_json/20180706_2dsted_60x_ch1.json',
    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch1.json',
    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch1.json'
]

overview_detector_kwargs = {"sigma": 2, "thresholds": [0.5, 0.5]}
detail_detector_kwargs = {"sigma": 3, "thresholds": [0.1, 0.1], "median_radius": 10}

In [ ]:
# init 3-level pipeline
pl = AcquisitionPipeline(save_path, ('overview', 'detail', 'sted'), save_combined_hdf5=True, name='spot-pair-pipeline')

# overview task generator: traverse in spiral with autofocus
atg_overview = AcquisitionTaskGenerator(
    "overview", 
    LocationRemover(JSONSettingsLoader(overview_settings_file)),
    SpiralOffsetGenerator(overview_spiral_move, get_current_stage_coords()),
    ScanModeSettingsGenerator('xyz'),
    FOVSettingsGenerator(fov_sizes["overview"], pixel_sizes["overview"]),
    StageOffsetsSettingsGenerator(SimpleFocusPlaneDetector(NewestDataSelector(pl, "overview")))
)

# spot pair detector in overview
detector = LegacySpotPairFinder(NewestDataSelector(pl, "overview"), **overview_detector_kwargs, plot_detections=True)

# detail task generation: settings from last overview + new FOV + grouped detections
atg_detail = AcquisitionTaskGenerator(
    "detail",
    LocationRemover(JSONSettingsLoader(detail_settings_file)),
    LocationKeeper(NewestSettingsSelector(pl, "overview")),
    ScanModeSettingsGenerator('xyz'),
    FOVSettingsGenerator(fov_sizes["detail"], pixel_sizes["detail"]),
    ScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector, fov_sizes["detail"]))
)

# spot pair detector in detail
# NOTE: we use PairedLegacySpotPairFinder, which will return the coordinates of both detected spots
detector_detail = PairedLegacySpotPairFinder(NewestDataSelector(pl, "detail"), **detail_detector_kwargs, plot_detections=True)

# super-resolution detail task generation: settings from last detail + new FOV
# do every measurement 3 times to assess reproducibility
atg_detail_more = AcquisitionTaskGenerator(
    "sted",
    LocationRemover( JSONSettingsLoader ( sted_settings_files, None, as_measurements=False)),
    LocationKeeper(NewestSettingsSelector(pl, "detail")),
    # 6 scan mode configurations (each orthogonal cut twice for different channels)
    ScanModeSettingsGenerator(['xy', 'xy', 'xz', 'xz', 'yz', 'yz'], as_measurements=False),
    FOVSettingsGenerator(fov_sizes["sted"], pixel_sizes["sted"]),
    # get 3x2 scan offsets (multiple configurations) based on detections
    # NOTE: we repeat detected pairs (coords_ch1, coords_ch2) 3 times to get (coords_ch1, coords_ch2, coords_ch1, coords_ch2, coords_ch1, coords_ch2) 
    MultipleScanOffsetsSettingsGenerator(ResultsRepeater(detector_detail, 3, True, True))
)

pl.add_stopping_condition(TimedStoppingCriterion(time_to_image))

# overview callback: re-add overview
pl.add_callback(atg_overview, "overview")
# overview callback 2: detect & do detail
pl.add_callback(atg_detail, "overview")
# detail callback: detect & do more detail
pl.add_callback(atg_detail_more, "detail")

# GO
pl.run(atg_overview)